In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
dest = '/content/drive/MyDrive/HSE Final Project/to_train'
name = 'submit_9_abandoned.csv'

df = pd.read_csv(dest + '/' + name)
df.head()

,shop_id,item_id,month,year,date_block_num,item_cnt,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,shop_city,shop_category,item_category_id,digital,category_type,category_subtype,item_cnt_lag_2,item_cnt_lag_3,item_cnt_lag_4,item_cnt_lag_5,item_cnt_lag_6,cnt_block_shop_lag_1,cnt_block_shop_lag_2,cnt_block_shop_lag_3,cnt_block_shop_lag_4,cnt_block_shop_lag_5,cnt_block_shop_lag_12,cnt_block_item_lag_1,cnt_block_item_lag_2,cnt_block_item_lag_3,cnt_block_item_lag_4,cnt_block_item_lag_5,cnt_block_item_lag_12,cnt_block_category_lag_1,cnt_block_category_lag_2,cnt_block_category_lag_3,cnt_block_category_lag_4,cnt_block_category_lag_5,cnt_block_category_lag_12,date_item_avg_item_cnt,shop_item_avg_item_cnt,cat_item_avg_item_cnt
0,0,19,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,40,False,7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.02222,0.0,0.02222
1,0,27,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,19,False,5,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.15550,0.0,0.05682
2,0,28,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,30,False,5,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.17770,0.0,0.14120
3,0,29,1,2013,0,0,0.0,0,0,0.0,0.0,29,4,23,False,5,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.08887,0.0,0.03738
4,0,32,1,2013,0,6,221.0,0,0,0.0,0.0,29,4,40,False,7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.64500,8.0,1.31900


In [ ]:
df['shop_id'] = df['shop_id'].astype(np.int8)
df['item_id'] = df['item_id'].astype(np.int16)
df['month'] = df['month'].astype(np.int8)
df['year'] = df['year'].astype(np.int16)
df['date_block_num'] = df['date_block_num'].astype(np.int8)
df['item_cnt'] = df['item_cnt'].astype(np.int32)
df['prev_value'] = df['prev_value'].astype(np.int32)
df['prev_value_year'] = df['prev_value_year'].astype(np.int32)
df['shop_city'] = df['shop_city'].astype(np.int8)
df['shop_category'] = df['shop_category'].astype(np.int8)
df['item_category_id'] = df['item_category_id'].astype(np.int8)
df['digital'] = df['digital'].astype(np.bool_)
df['category_type'] = df['category_type'].astype(np.int8)
df['category_subtype'] = df['category_subtype'].astype(np.int8)
df['item_cnt_lag_2'] = df['item_cnt_lag_2'].astype(np.int32)
df['item_cnt_lag_3'] = df['item_cnt_lag_3'].astype(np.int32)
df['item_cnt_lag_4'] = df['item_cnt_lag_4'].astype(np.int32)
df['item_cnt_lag_5'] = df['item_cnt_lag_5'].astype(np.int32)
df['item_cnt_lag_6'] = df['item_cnt_lag_6'].astype(np.int32)

In [ ]:
for col in df.columns[22:40]:
    df[col] = df[col].astype(np.int32)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10913804 entries, 0 to 10913803
Data columns (total 43 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   shop_id                    int8   
 1   item_id                    int16  
 2   month                      int8   
 3   year                       int16  
 4   date_block_num             int8   
 5   item_cnt                   int32  
 6   item_price                 float64
 7   prev_value                 int32  
 8   prev_value_year            int32  
 9   stock_prev                 float64
 10  stock_prev_year            float64
 11  shop_city                  int8   
 12  shop_category              int8   
 13  item_category_id           int8   
 14  digital                    bool   
 15  category_type              int8   
 16  category_subtype           int8   
 17  item_cnt_lag_2             int32  
 18  item_cnt_lag_3             int32  
 19  item_cnt_lag_4             int32  
 20  

Due to the fact that we predict future, it makes no sense to shuffle the data

In [ ]:
# Indices for RandomizedSearchCV
index = df.index

condition_train = df.date_block_num < 33
train_indices = index[condition_train]

condition_valid = df.date_block_num == 33
valid_indices = index[condition_valid]

train_indices = train_indices.tolist()
valid_indices = valid_indices.tolist()

print(train_indices[-5:])
print(valid_indices[-5:])

[10675627, 10675628, 10675629, 10675630, 10675631]
[10913799, 10913800, 10913801, 10913802, 10913803]


In [ ]:
import multiprocessing
multiprocessing.cpu_count()

4

In [ ]:
# del df
# learning_rate = 0.1
# max_depth = 8
# n_estimators = 1000

In [ ]:
# eval_set = [(X_train, y_train), (X_valid, y_valid)]

In [ ]:
X = df.drop(['item_cnt'], axis=1)
y = df['item_cnt']

In [ ]:
del df

RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

custom_cv = [(train_indices, valid_indices)]
params = {'learning_rate': [0.05, 0.1, 0.3], 
          'max_depth': [7, 9, 11],
          'min_child_weight': [0, 5, 10, 20],
          'colsample_bytree': [0.7, 0.9, 1],
          'n_estimators': [300, 500, 1000]}

fit_params={"early_stopping_rounds": 10,
            "eval_metric": "rmse", 
            "eval_set": [(X.loc[valid_indices], y.loc[valid_indices])],
            "verbose": 2}

In [ ]:
random_state = 13
verbosity = 2
obj = 'reg:squarederror'

In [ ]:
xgbr = xgb.XGBRegressor(verbosity=verbosity,
                        objective=obj,
                        random_state=random_state)

In [ ]:
# X_train = df[df.date_block_num < 33].drop(['item_cnt'], axis=1)
# y_train = df[df.date_block_num < 33]['item_cnt']
# X_valid = df[df.date_block_num == 33].drop(['item_cnt'], axis=1)
# y_valid = df[df.date_block_num == 33]['item_cnt']

In [ ]:
# xgbr.fit(X_train, y_train,
#          eval_metric=['rmse', 'rmse'], eval_set=eval_set,
#          early_stopping_rounds=15, verbose=True)

**Good Params:**

*   Max_depth: 7 < x <= 9 (the biggest 10)
*   Min_child_weight: x >= 5
*   col_sample_by_tree: 0.7 <= x <= 0.9
*   eta: x >= 0.05
*   n_estimators < 1000


---


The best model yet:
colsample_bytree=0.9, learning_rate=0.05, max_depth=9, min_child_weight=5, n_estimators=300;, score=-4.012



P.S. After a lot of time-consuming trainings these params were chosen.

In [ ]:
grid_obj_xgbr = RandomizedSearchCV(xgbr, params,
                                   cv=custom_cv,
                                   n_iter=5,
                                   scoring='neg_root_mean_squared_error',
                                   verbose=5, n_jobs=1)

In [ ]:
grid_obj_xgbr.fit(X, y, **fit_params)

Fitting 1 folds for each of 5 candidates, totalling 5 fits
[11:43:35] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[11:45:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=7
[0]	validation_0-rmse:5.18832
Will train until validation_0-rmse hasn't improved in 10 rounds.
[11:47:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 0 pruned nodes, max_depth=7
[11:48:34] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 0 pruned nodes, max_depth=7
[2]	validation_0-rmse:4.9376
[11:50:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=7
[11:51:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots

KeyboardInterrupt: ignored

[0]	validation_0-rmse:5.25962
Will train until validation_0-rmse hasn't improved in 10 rounds.
[20:03:57] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 412 extra nodes, 0 pruned nodes, max_depth=9
[20:05:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 566 extra nodes, 0 pruned nodes, max_depth=9
[2]	validation_0-rmse:5.1193
[20:06:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 538 extra nodes, 0 pruned nodes, max_depth=9


In [ ]:
print("\n The best estimator across ALL searched params:\n", grid_obj_xgbr.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_obj_xgbr.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_obj_xgbr.best_params_)

In [ ]:
import matplotlib.pyplot as plt

# retrieve performance metrics
results = xgbr.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)

# plot rmse
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
ax.legend()
plt.ylabel('RMSE')
plt.title('XGBoost RMSE')
plt.show()

In [ ]:
import pickle

dest = '/content/drive/MyDrive/HSE Final Project/Models/'
file_name = 'xgb_reg_5 (new valid + prices).pkl'

# save
pickle.dump(xgbr, open(dest + file_name, 'wb'))

In [ ]:
import pickle

dest = '/content/drive/MyDrive/HSE Final Project/Models/'
file_name = 'xgb_reg_5 (new valid + prices).pkl'
# load
xgbr = pickle.load(open(dest + file_name, "rb"))

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = xgbr.predict(X_valid)
y_pred_rounded = np.around(y_pred)
mse = mean_squared_error(y_valid, y_pred)
mse_rd = mean_squared_error(y_valid, y_pred_rounded)

print('MSE: %.2f' % mse)
print('RMSE: %.2f' % (mse ** (0.5)))

print('MSE (rounded): %.2f' % mse_rd)
print('RMSE (rounded): %.2f' % (mse_rd ** (0.5)))

In [ ]:
# from sklearn.metrics import mean_squared_log_error
# msle = mean_squared_log_error(y_test, y_pred)
# print('RMSLE: %.2f' % (msle ** (0.5)))

In [ ]:
dest = '/content/drive/MyDrive/HSE Final Project/to_test'
name = 'submit_9_test.csv'

test_df = pd.read_csv(dest + '/' + name)
test_df.head()

In [ ]:
test_pred = xgbr.predict(test_df)
print(test_pred)

In [ ]:
name_sample = 'sample_submission.csv'

sample = pd.read_csv(dest + '/' + name_sample)
sample.head()

In [ ]:
sample.item_cnt_month = np.around(test_pred)
sample.head()

In [ ]:
dest = '/content/drive/MyDrive/HSE Final Project/submissions/'
sample.to_csv(dest + 'submission #17 (new prices + valid).csv', index=False)

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt


def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [ ]:
plot_features(xgbr, (10, 14))